In [2]:
import tensorflow as tf
import tensorflow.keras
import time

 - 파이프라인이란, 일종의 데이터 흐름이다. 잘 구축해두면, 데이터만 바꾸면 싹 다 바뀐다.
 
### prefetch 개념

미리 땡겨오는 개념이다.

In [27]:
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

model = Sequential([
    GlobalAveragePooling2D(input_shape = (4, 4, 3))
])

In [28]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_6 ( (None, 3)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [5]:
import numpy as np

In [33]:
x = np.array([[[1, 2, 3, ],
         [1, 2, 3, ],
         [1, 2, 3, ],
         [1, 2, 3, ]],
            [[1, 2, 3, ],
         [1, 2, 3, ],
         [1, 2, 3, ],
         [1, 2, 3, ]],
            [[1, 2, 3, ],
         [1, 2, 3, ],
         [1, 2, 3, ],
         [1, 2, 3, ]],
             [[1, 2, 3, ],
         [1, 2, 3, ],
         [1, 2, 3, ],
         [1, 2, 3,]]])
x = x[np.newaxis, ...]
x.shape

(1, 4, 4, 3)

In [35]:
np.squeeze(model.predict(x))

array([1., 2., 3.], dtype=float32)

### Sobel filter 보기

In [46]:
from tensorflow.keras.layers import Conv2D, GlobalMaxPool2D
import tensorflow_datasets as tfds

In [42]:
mnist = tfds.load('mnist',
                  as_supervised=True,
                  with_info=True,
                  split = tfds.Split.TRAIN.subsplit(weighted = (7, 2, 1)))

In [44]:
(train_input, valid_input, test_input), metadata = mnist

In [66]:
def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1 # -1 ~ 1 사이로 스케일링 해준다. 하이퍼볼릭 탄젠트 방식.
    return image, label

In [67]:
train_data = train_input.map(format_example)

In [49]:
model = Sequential([
    GlobalMaxPool2D(input_shape = (28, 28, 1))
])

In [50]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_max_pooling2d (Global (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


### Dense 1개

In [109]:
tf.kernel

AttributeError: module 'tensorflow' has no attribute 'kernel_'

In [117]:
 model = Sequential([
     Dense(1, input_shape = (3, 3), # 데이터가 1개일 때 input_shape을 적는다.
           use_bias = False,
#            kernel_initializer = tf.keras.initializers.he_normal(seed=None), # 가중치 초기화
           kernel_initializer = tf.keras.initializers.ones())
 ])

In [165]:
 model = Sequential([
     Dense(2, input_shape = (3, 3),
          kernel_initializer = 'ones'),
     Dense(2, kernel_initializer = 'ones'),
     ReLU(),
     Flatten()
 ])

In [166]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_38 (Dense)             (None, 3, 2)              8         
_________________________________________________________________
dense_39 (Dense)             (None, 3, 2)              6         
_________________________________________________________________
re_lu (ReLU)                 (None, 3, 2)              0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6)                 0         
Total params: 14
Trainable params: 14
Non-trainable params: 0
_________________________________________________________________


In [232]:
a = np.arange(9).reshape(1, 3, 3)
b = np.arange(18).reshape(2, 3, 3)
c = np.arange(24).reshape(2, 4, 3)

In [137]:
# b = np.array([[[1, 2, 3],[1, 2, 3],[1, 2, 3]],
#              [[1, 2, 3], [1, 2, 3], [1, 2, 3]]])
b

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]]])

In [160]:
model.predict(b)

array([[ 6.,  6., 24., 24., 42., 42.],
       [60., 60., 78., 78., 96., 96.]], dtype=float32)

In [98]:
model.predict(c).shape

(2, 4, 1)

곱하는 애들은 행렬곱이기 때문에 첫 번재 행의 모양을 따라가는 것이다. 그래서 Dense shape의 모양이 변한다.

Flatten은 레코드 구성방식 C방식에 따라서, 로우마다 짤라서 붙여준다. ★

# 모바일넷 컨볼루션 레이어

In [148]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

model = MobileNetV2(include_top = False,
                    input_shape = (32, 32, 3))

In [146]:
model.summary() # sequence 모델이 아니기 때문에, connected to라는 것이 있다.

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 33, 33, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

block_4_expand_relu (ReLU)      (None, 4, 4, 192)    0           block_4_expand_BN[0][0]          
__________________________________________________________________________________________________
block_4_depthwise (DepthwiseCon (None, 4, 4, 192)    1728        block_4_expand_relu[0][0]        
__________________________________________________________________________________________________
block_4_depthwise_BN (BatchNorm (None, 4, 4, 192)    768         block_4_depthwise[0][0]          
__________________________________________________________________________________________________
block_4_depthwise_relu (ReLU)   (None, 4, 4, 192)    0           block_4_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_4_project (Conv2D)        (None, 4, 4, 32)     6144        block_4_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_4_pr

__________________________________________________________________________________________________
block_8_project (Conv2D)        (None, 2, 2, 64)     24576       block_8_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_8_project_BN (BatchNormal (None, 2, 2, 64)     256         block_8_project[0][0]            
__________________________________________________________________________________________________
block_8_add (Add)               (None, 2, 2, 64)     0           block_7_add[0][0]                
                                                                 block_8_project_BN[0][0]         
__________________________________________________________________________________________________
block_9_expand (Conv2D)         (None, 2, 2, 384)    24576       block_8_add[0][0]                
__________________________________________________________________________________________________
block_9_ex

__________________________________________________________________________________________________
block_13_expand_BN (BatchNormal (None, 2, 2, 576)    2304        block_13_expand[0][0]            
__________________________________________________________________________________________________
block_13_expand_relu (ReLU)     (None, 2, 2, 576)    0           block_13_expand_BN[0][0]         
__________________________________________________________________________________________________
block_13_pad (ZeroPadding2D)    (None, 3, 3, 576)    0           block_13_expand_relu[0][0]       
__________________________________________________________________________________________________
block_13_depthwise (DepthwiseCo (None, 1, 1, 576)    5184        block_13_pad[0][0]               
__________________________________________________________________________________________________
block_13_depthwise_BN (BatchNor (None, 1, 1, 576)    2304        block_13_depthwise[0][0]         
__________

### sequential에 안 넣고 model 도전

In [223]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, ReLU, Conv2D, BatchNormalization, MaxPool2D, GlobalAvgPool2D


input_layer = Input(shape = (32, 32))
x1 = Dense(2, kernel_initializer = 'ones')(input_layer)

model2 = Model(input_layer, x1)

In [154]:
model2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 32, 32)]          0         
_________________________________________________________________
dense_28 (Dense)             (None, 32, 2)             66        
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________


★★★ activatoin 함수를 따로 빼서 쓰는 이유는, 디버깅하기 쉽기 때문이다. summary했을 때, dense 내부 activatoin함수를 모르기 때문에 밖으로 빼서 아는 기법이다.

### Conv2d

In [269]:
a = np.arange(25).reshape(5, 5)
b = np.ones((3, 3))
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [278]:
row = a.shape[0] - b.shape[0] + 1
for i in range(3):
    print(i)

0
1
2


In [275]:
temp = []

for i in range(a.shape[0] - b.shape[0] + 1):
    for j in range(a.shape[0] - b.shape[0] + 1):
        temp.append(np.sum(a[i:i+3, j:j+3]*b))
        
# np.array(temp).mean(axis = 0)
np.array(temp).reshape(3, 3)

array([[ 54.,  63.,  72.],
       [ 99., 108., 117.],
       [144., 153., 162.]])

In [225]:
model = Sequential([
    Conv2D(2, (3, 3), 
           kernel_initializer = 'ones',
           input_shape = (10, 10, 1)),
    GlobalAvgPool2D(), # 각 필터를 평균잡는 것
#     Conv2D(2, (3, 3), 
#            kernel_initializer = 'ones'),
#     BatchNormalization()
])

In [226]:
model.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 8, 8, 2)           20        
_________________________________________________________________
global_average_pooling2d_7 ( (None, 2)                 0         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [231]:
w, b = model.layers[0].get_weights()

In [241]:
t = model.predict(a)
t[..., 0].mean()
t

array([[445.5, 445.5]], dtype=float32)

In [189]:
for img, label in train_data.take(1):
    pass

img

<tf.Tensor: id=21856, shape=(28, 28, 1), dtype=float32, numpy=
array([[[-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ]],

       [[-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [-1.   